---
title: Selecting Rat genes for Enformer CAGE predictions
description: We are looking for genes with (1) variation in observed gene expression across individuals, (2) high prediction performance in elastic net training, and (3) multiple causal variants.
author: Sabrina Mi
date: 8/10/23
---

## Calculate gene expression variance across individuals

In [32]:
import pandas as pd
import numpy as np

In [7]:
## Read in expression data
tpm = pd.read_csv("/home/s1mi/enformer_rat_data/Brain.rn7.expr.tpm.bed", header = 0, sep="\t",  index_col= 'gene_id')
iqn = pd.read_csv("/home/s1mi/enformer_rat_data/Brain.rn7.expr.iqn.bed", header = 0, sep="\t",  index_col= 'gene_id')

/tmp/ipykernel_123362/1386399847.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  tpm = pd.read_csv("/home/s1mi/enformer_rat_data/Brain.rn7.expr.tpm.bed", header = 0, sep="\t",  index_col= 'gene_id')
/tmp/ipykernel_123362/1386399847.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  iqn = pd.read_csv("/home/s1mi/enformer_rat_data/Brain.rn7.expr.iqn.bed", header = 0, sep="\t",  index_col= 'gene_id')


In [33]:
tpm_var = tpm.iloc[:, 3:].var(axis=1)
np.average(tpm_var)

1003.8945129200853

In [34]:
iqn_var = iqn.iloc[:, 3:].var(axis=1)
np.average(iqn_var)

0.6306594759954833

We first subset to genes in the top decile for both TPM and IQN variance.

In [37]:
tpm_threshold = tpm_var.quantile(0.9)
iqn_threshold = iqn_var.quantile(0.9)
high_tpm_var_genes = set(tpm[tpm_var> tpm_threshold].index)
high_iqn_var_genes = set(iqn[iqn_var> iqn_threshold].index)
high_var_genes = high_tpm_var_genes.intersection(high_iqn_var_genes)
print(len(high_var_genes), "genes with high variance")

167 genes with high variance
